In [1]:
#! python
# -*- coding: latin-1 -*-
import os
from guessit import guess_file_info
import codecs
from datetime import datetime
import enzyme
import json
import re
import math
from ffprobe import FFProbe
from avMetadataInfo import mediaObject
import time
import locale, unicodedata
import imdb
import pickle

In [2]:
def userAgrees(p1, p2):
    while True:
        in_ = raw_input(u"{}  is {} ? ".format(p1, p2))
        if  in_ == "" or in_ == "True" or in_ == "true" or in_ == "yes" or in_ == "y" or in_ == "Y" or in_ == "Yes":
            return True
        elif in_ == "False" or in_ == "false" or in_ == "no" or in_ == "n" or in_ == "N" or in_ == "No":
            return False

In [3]:
def convertSize(size):
    s = round(float(size)/long(1073741824), 2)
    return '%s %s' % (s,"GB")
#    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
#    i = int(math.floor(math.log(size,1024)))
#    p = math.pow(1024,i)
#    s = round(size/p,2)
#    if (s > 0):
#        return '%s %s' % (s,size_name[i])
#    else:
#        return '0B'

In [4]:
def getUser(path):
    with open(path, 'r') as f:
        a = f.read().strip()
    data = a.split(';')
    return (data[0], data[1])

In [5]:
def sec2hours(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return "%02d:%02d:%02d" % (h, m, s)

In [6]:
def get_files(folder, checked_files=[], file_types=[".mp4", ".mkv", ".avi"], min_size=long(100000000)):
    files = []
    for root, dirs, _files in os.walk(folder):
        for name in _files:
            if os.path.join(root, name) not in checked_files and name not in checked_files:
                for ftype in file_types:
                    if name.endswith(ftype):
                        fsize = os.path.getsize(os.path.join(root, name))
                        if fsize > min_size:
                            files.append((os.path.join(root, name), convertSize(fsize), name))
                            checked_files.append(name)
                        break
    
    return files

In [7]:
def get_data(files, min_runtime=5):
    prog = re.compile('sample', flags=re.IGNORECASE)
    notwatched = re.compile(u'não visto', flags=re.IGNORECASE)
    jdata = []
    data_keys = ['container', 'screenSize', 'format', 'releaseGroup', 'year']
    json_keys = {'container' : "fileType", 'screenSize' : "resolution", 'format' : "format", 'releaseGroup' : "releaseGroup", 'year' : 'year'}
    swipe = []
    for count in xrange(len(files)):
        pair = files[count]
        x = pair[0]
        try:
            tmp = guess_file_info(x, type='movie')
        except UnicodeDecodeError as e:
            normalized_x = unicodedata.normalize('NFKD', x).encode(locale.getpreferredencoding(),'ignore')
            tmp = guess_file_info(normalized_x, type='movie')
        tdic = {}
        try:
            title = tmp['title']
        except KeyError as e:
            print x
            print tmp.nice_string()
        try:
            rg = tmp['releaseGroup']
            aws = prog.search(rg)
            if aws:
                swipe.append(count)
                continue
        except KeyError:
            pass

        tdic['movie'] = title
        tdic['fileName'] = pair[2]
        tdic['fileSize'] = pair[1]
        for key in data_keys:
            try:
                tdic[json_keys[key]] = tmp[key]
            except KeyError:
                tdic[json_keys[key]] = ""
        

        metadata = mediaObject(pair[0].encode(locale.getpreferredencoding()))
        if metadata.getVideoCodec() is None:
            swipe.append(count)
            continue
            
        tlenght = time.strptime(metadata.getTotalRunningTimeRaw()[:-3], "%H:%M:%S")
        
        try:
            tdic['bitrateKbps'] = metadata.getVideoBitRate()/1000
        except ValueError:
            tdic['bitrateKbps'] = 8*metadata.getFileSize()/(1000 * (tlenght.tm_hour * 3600 + tlenght.tm_min *60 + tlenght.tm_sec))
        
        tdic['height'] = metadata.getVideoResolutionHeight()
        tdic['width'] = metadata.getVideoResolutionWidth()
        tdic['runtime'] = tlenght.tm_hour * 60 + tlenght.tm_min + math.trunc(float(tlenght.tm_sec)/60)       
        if tdic['runtime'] < min_runtime:
            swipe.append(count)
            continue
        wreg = notwatched.search(x)
        if wreg:
            tdic['watched'] = False
        else:
            tdic['watched'] = True
        
        jdata.append(tdic)
    print swipe
    return jdata, swipe

In [8]:
folder = u"C:/Users/Usuario/Downloads/Filmes"
files = get_files(folder)
print len(files)
files += get_files(u"F:/Backup/Bibliotecas/Filmes/Vistos") 
print len(files)
files += get_files(u"F:/Backup/Bibliotecas/Filmes/Não Vistos") 
print len(files)
files += get_files(u"F:/Backup/Bibliotecas/Filmes/wa")
print len(files)
(jdata, swipe) = get_data(files)

c = 0
for pos in swipe:
    del files[pos+c]
    c -= 1
print len(files)

11
177
205
246
[]
246


In [9]:
used = []
manual_id = {}
u_agree_db = {}
ia = imdb.IMDb() # by default access the web.

In [25]:
for i in xrange(len(jdata)):
    if i in used:
        continue
    movie = jdata[i]
    mtitle = movie['movie']
    year = movie['year']
    item = None
    try:
        items = ia.search_movie(u"{} {}".format(mtitle, year), 6)
        for sitem in items:
            if sitem['kind'] == 'movie':
                item = sitem
                break
        if item is None:
            raise IndexError
        try:
            flag = u_agree_db[files[i][2]]
        except KeyError:
            flag = userAgrees(files[i][2], item['long imdb title'])
            u_agree_db[files[i][2]] = flag
        if not flag:
            raise IndexError
    except IndexError:
        if files[i][2] in manual_id:
            item = ia.get_movie(manual_id[files[i][2]])
        else:
            print u"I couldn't find the data for {}, please manually enter imdbID".format(files[i][0])
            in_ = raw_input("IMDB id: ").lstrip("tt")
            manual_id[files[i][2]] = in_
            item = ia.get_movie(in_)
        print mtitle, "\t", item['long imdb title']
    ia.update(item)
    jdata[i]['movieImdb'] = item['long imdb title']
    jdata[i]['movieLink'] = u'<a title="{}" href="{}" target="_blank">{}</a>'.format(jdata[i]['movieImdb'], ia.get_imdbURL(item), jdata[i]['movieImdb'])
    jdata[i]['id'] = item.movieID
    jdata[i]['year'] = int(item['year'])
    try:
        jdata[i]['rating'] = item['rating']
    except KeyError:
        jdata[i]['rating'] = 0
    used.append(i)

Amour 	Amour (2012)
Argo 	Argo (2012)
Beginners 	Beginners (2010)
Beirut 	Cheap Magic Inside (2011)
Birdman (2014) [1080p, 1916x1032, AVC, WEB-DL, RARBG].mkv  is Birdman: Or (The Unexpected Virtue of Ignorance) (2014) ? 
Black Swan 	Black Swan (2010)
Black Swan 	Black Swan (2010)
I couldn't find the data for F:/Backup/Bibliotecas/Filmes/Vistos\Cashback [2006, 720p]\Cashback (2006) by Sean Ellis [720p, 1280x544, x264, RELEASE].mp4, please manually enter imdbID
IMDB id: tt0460740
Cashback 	Cashback (2006)
Cidade de Deus (2002) by Fernando Meirelles [720p, 1280x692, x264, BluRay, CtrlHD].mkv  is City of God (2002) ? 
Control 	Control (2007)
Creation 	Creation (I) (2009)
Ex Machina (2015) [1080p, 1920x808, x264, BRRip, JYK].mkv  is Ex Machina (2015) ? 
Gravity 	Gravity (2013)
Interstellar (2014) [1080p, 1920x1080, x264, BluRay, YIFY].mp4  is Interstellar (2014) ? 
Jeux d'enfants 	Jeux d'enfants (2003)
Kingsman.The.Secret.Service.2014.1080p.BluRay.x264.YIFY.mp4  is Kingsman: The Secret Serv

In [88]:
jdata_sorted = sorted(jdata, key= lambda x:x['movieImdb'])
aa = json.dumps(jdata_sorted, sort_keys = True, indent = 4, ensure_ascii=False)
json.dump(jdata_sorted, codecs.open('json_data/mvdata.json', 'w', 'utf-8'), sort_keys = True, indent = 4, ensure_ascii=False)
json.dump(jdata_sorted, codecs.open('../../movie_html/json_data/mvdata.json', 'w', 'utf-8'), sort_keys = True, indent = 4, ensure_ascii=False)

In [89]:
pickle.dump(manual_id, open("manual_id.bak", "wb"))
pickle.dump(u_agree_db, open("u_agree_db.bak", "wb"))
pickle.dump(jdata, open("jdata.bak", "wb"))

In [10]:
manual_id = pickle.load(open("manual_id.bak", "rb"))
u_agree_db= pickle.load(open("u_agree_db.bak", "rb"))

In [14]:
i

24

In [21]:
files[i]

(u'F:/Backup/Bibliotecas/Filmes/Vistos\\Amour [2012, 360p]\\Amour (2012) by Michael Haneke [360p, 688x368, XviD, BRRip, HORiZON].avi',
 '1.78 GB',
 u'Amour (2012) by Michael Haneke [360p, 688x368, XviD, BRRip, HORiZON].avi')

In [24]:
manual_id[files[i][2]]

'1602620'

In [23]:
manual_id

{u'Amour (2012) by Michael Haneke [360p, 688x368, XviD, BRRip, HORiZON].avi': '1602620',
 u'Argo (2012) by Ben Affleck [720p, 1280x528, x264, BluRay, YIFY].mp4': '1024648',
 u'Beginners (2010) by Mike Mills [720p, 1280x688, x264, BRRip, YIFY].mp4': '1532503',
 u'Beirut - Cheap Magic Inside (2010) by Vincent Moon [480p, 720x400, XviD, ].avi': '4272988',
 u'Black Swan (2010) by Darren Aronofsky [360p, 640x272, XviD, DVDRip, ] CD1.avi': '0947798',
 u'Black Swan (2010) by Darren Aronofsky [360p, 640x272, XviD, DVDSCR, ] CD2.avi': '0947798',
 u'Control (2007) by Anton Corbijn [720p, 1280x544, x264, BluRay, HANGOVER].mkv': '0421082',
 u'Creation (2009) by Jon Amiel [360p, 624x256, XviD, DVDRip, ].avi': '0974014',
 u'Fanny och Alexander (1982) by Ingmar Bergman [720p, 1280x720, DivX, ].avi': '0083922',
 u'Gravity (2013) by Alfonso Cuar\xf3n [1080p, 1918x798, AVC, WEB-DL, PublicHD].mkv': '1454468',
 u"Jeux d'enfants (2003) by Yann Samuell [360p, 576x304, XviD, DVDRip, ].avi": '0364517',
 u'Kid